# Discrete

In [ ]:
from glob import glob
from gnwrapper import Animation
import gym
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
# Constants.
ENV_NAME = "CartPole-v0"
SEED = 123
STEPS = int(1e3)
VISUALIZE = True
WEIGHTS = f"../data/dqn_{ENV_NAME}_weights.h5f"

In [ ]:
# Build environment.
env = Animation(gym.make(ENV_NAME))
np.random.seed(SEED)
env.seed(SEED)
nb_actions = env.action_space.n

In [ ]:
# Build model.
model = Sequential()

model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

model.summary()

In [ ]:
# Build agent.
memory = SequentialMemory(limit=STEPS, window_length=1)
policy = BoltzmannQPolicy()

agent = DQNAgent(
    model=model,
    nb_actions=nb_actions,
    memory=memory,
    nb_steps_warmup=10,
    target_model_update=1e-2,
    policy=policy,
)

agent.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
# Load weights if exist.
if glob(WEIGHTS + "*"):
    agent.load_weights(WEIGHTS)

In [ ]:
# Train.
env = Animation(gym.make(ENV_NAME))
agent.fit(env, nb_steps=STEPS, visualize=VISUALIZE, verbose=2)

In [ ]:
# Save weights..
agent.save_weights(WEIGHTS, overwrite=True)

In [ ]:
# Evaluate.
env = Animation(gym.make(ENV_NAME))
agent.test(env, nb_episodes=5, visualize=VISUALIZE)